# Initialization

### Mount Drive

In [ ]:
#@title Set up Directory { run: "auto"}
import os
from IPython.display import clear_output
from google.colab import drive 
from IPython.display import clear_output
drive.mount('/content/gdrive')
working_directory = 'My Drive/Sem8/COVID19_Tweets' #@param {type:"string"}
wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
current directory is : /content/gdrive/.shortcut-targets-by-id/1_doUiOLFMGIDBhRxyDqTP0GhjehF9G6P/Sem8/COVID19_Tweets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install twarc

In [ ]:
%pip install twarc
%pip install jsonlines
%pip install wget
clear_output()

In [ ]:
#Check if TWARC was installed correctly on the Virtual Machine
%pip show twarc
%pip show jsonlines

Name: twarc
Version: 2.0.13
Summary: Archive tweets from the command line
Home-page: https://github.com/docnow/twarc
Author: Ed Summers
Author-email: ehs@pobox.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: requests-oauthlib, python-dateutil, click, click-plugins, click-config-file
Required-by: 
Name: jsonlines
Version: 2.0.0
Summary: Library with helpers for the jsonlines file format
Home-page: https://github.com/wbolster/jsonlines
Author: Wouter Bolsterlee
Author-email: wouter@bolsterl.ee
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 


### Twitter API Keys

In [ ]:
#@title Insert API Keys here { run : "auto"}
from twarc import Twarc

# These keys are received after applying for a twitter developer account

consumer_key = "" #@param {type:"string"}
consumer_secret = "" #@param {type:"string"}
access_token = "" #@param {type:"string"}
access_token_secret = "" #@param {type:"string"}

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

### Download Github Files onto Drive for given dates

You need to frist indicate the date range of file you would like to hydrate. Rememeber that the Start_date cannot be before 2020-01-22. If a incorrect end date is selected it will be not considered.  At the end you will see how many files you are requesting to download 

(it is assumed you want all hours from a given day, if you dont want some hour, you will need to deleted the hours you wont want).


In [ ]:
#@title Date fields
import pandas as pd
Start_date = '2021-04-15' #@param {type:"date"}
#@title Date fields
End_date = '2021-04-15' #@param {type:"date"}



datelist = pd.date_range(Start_date, End_date).tolist()
number_files=0
for date in datelist:
  for h in ['00','01', '02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']:
    number_files=number_files+1

print("You have requested to download a total of "+ str(number_files) + ' files \n The more files you download at once the longer it will take to upload to your Drive')

You have requested to download a total of 24 files 
 The more files you download at once the longer it will take to upload to your Drive


The code below will create a new directory `Summary_Details_files` and save all the `.csv. files from the reposiory that were requested on that directory.

In [ ]:

from datetime import datetime
import wget

path = os.path.join(wd, "Summary_Details_files") 
if not os.path.exists('./Summary_Details_files'):
  try:
    os.makedirs(path)
    os.chdir('Summary_Details_files')
  except:
    pass

datelist = pd.date_range(Start_date, End_date).tolist()
files_list=[]
for date in datelist:
  day=date.strftime("%Y_%m_%d")
  p1="https://raw.githubusercontent.com/lopezbec/COVID19_Tweets_Dataset/master/Summary_Details/"
  m=day[0:7]
  for h in ['00','01', '02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']:
    file=p1+m+"/"+day+"_"+str(h)+"_Summary_Details.csv"
    wget.download(file)


# Choose Settings Tweets_IDs

This are all the files you requested:

In [ ]:
import os
import pandas as pd 

Summary_Details=pd.DataFrame() 
directory =  os.getcwd()
for root,dirs,files in os.walk(directory):
  for file in files:
    if file.endswith(".csv"):
      data = pd.read_csv(file) 
      frames = [Summary_Details, data]
      Summary_Details = pd.concat(frames)
Summary_Details

,Tweet_ID,Language,Geolocation_coordinate,RT,Likes,Retweets,Country,Date Created
0,1371250205094477825,ar,NO,NO,1,10,NaN,Mon Mar 15 00:01:40 +0000 2021
1,1371251482595581955,en,NO,NO,0,0,NaN,Mon Mar 15 00:06:44 +0000 2021
2,1371251959840309248,ar,NO,NO,3,24,NaN,Mon Mar 15 00:08:38 +0000 2021
3,1371249785118859267,en,NO,YES,0,32161,NaN,Mon Mar 15 00:00:00 +0000 2021
4,1371249785592639490,en,NO,YES,0,10864,NaN,Mon Mar 15 00:00:00 +0000 2021
...,...,...,...,...,...,...,...,...
176934,1371612155666120704,en,NO,NO,0,0,NaN,Mon Mar 15 23:59:55 +0000 2021
176935,1371612163882766337,en,NO,YES,0,101,NaN,Mon Mar 15 23:59:57 +0000 2021
176936,1371612168538439683,en,NO,NO,0,0,NaN,Mon Mar 15 23:59:58 +0000 2021
176937,1371612171541622788,en,NO,NO,0,0,NaN,Mon Mar 15 23:59:59 +0000 2021


### Filter Tweets


You can filter the tweets by:


In [ ]:

Just_English_Tweets= True #@param {type:"boolean"}
Just_with_Geolocation_information = False #@param {type:"boolean"}
Just_Original_tweets = True #@param {type:"boolean"}
Minimun_No_Likes =  5#@param {type:"number"}
Minimun_No_Retweets =  0#@param {type:"number"}

Summary_Details2=Summary_Details
if(Just_English_Tweets):
  Summary_Details2=Summary_Details2[ Summary_Details2['Language']=='en']
if(Just_with_Geolocation_information):
  Summary_Details2=Summary_Details2[ Summary_Details2['Geolocation_coordinate']=='YES']
if(Just_Original_tweets):
  Summary_Details2=Summary_Details2[ Summary_Details2['RT']=='NO']


Summary_Details2=Summary_Details2[ Summary_Details2['Likes']>=Minimun_No_Likes]
Summary_Details2=Summary_Details2[ Summary_Details2['Retweets']>=Minimun_No_Retweets]

Summary_Details2

,Tweet_ID,Language,Geolocation_coordinate,RT,Likes,Retweets,Country,Date Created
52,1371249788922994690,en,NO,NO,10,3,NaN,Mon Mar 15 00:00:00 +0000 2021
125,1371249805964500992,en,NO,NO,26,10,NaN,Mon Mar 15 00:00:04 +0000 2021
140,1371249809101836291,en,NO,NO,30,3,NaN,Mon Mar 15 00:00:05 +0000 2021
202,1371249821932253196,en,NO,NO,11,1,NaN,Mon Mar 15 00:00:08 +0000 2021
268,1371249835014287362,en,NO,NO,15,1,NaN,Mon Mar 15 00:00:11 +0000 2021
...,...,...,...,...,...,...,...,...
175968,1371610520776761350,en,NO,NO,6,1,NaN,Mon Mar 15 23:53:26 +0000 2021
176110,1371610747088736260,en,NO,NO,7,1,NaN,Mon Mar 15 23:54:20 +0000 2021
176350,1371611165231489027,en,NO,NO,5,0,NaN,Mon Mar 15 23:55:59 +0000 2021
176872,1371612066331697157,en,NO,NO,9,0,NaN,Mon Mar 15 23:59:34 +0000 2021


For more personalized filtering you can modify the pandas dataframe directly

### Save configuration into a file
All the IDs are read into a single set in the previous code block using the specified configuration. The ID Output file stores all the IDs in a single file so that the configuration blocks don't have to be run again. In case the program unexpectedly stops, you can just run the code for Initialization and then the code for Hydration.

In [ ]:
#@title Enter ID output file {run: "auto"}
os.chdir(wd)
final_tweet_ids_filename = "final_ids.txt" #@param {type: "string"}
# The set of IDs is stored in this file.
with open(final_tweet_ids_filename, "w+") as f:
    for id in Summary_Details2['Tweet_ID']:
        f.write('%s\n' % id)

# Hydrate

### Set up output file

The final_tweet_ids_filename should be exactly the same as the ID output file from the Configuration block. If this file does not exist in the working directory, you have to re-run the Configuration block.

Please also keep the output_filename the same in case the code is halted. That way, tweets already hydrated aren't re-hydrated for no reason. 

Also, please do not remove the .txt file created after running the Hydrate block until all the data is converted to a CSV file.

In [ ]:
#@title Set up Directory { run: "auto"}
final_tweet_ids_filename = "final_ids.txt" #@param {type: "string"}
output_filename = "mar2.csv" #@param {type: "string"}

The time for this code will depend on how many tweets you want to “hydrate”. Also, be advise of the Tweet API limit, the code will “go to sleep” once the limit is reach and automatically continue. 
You can leave this code running in Google Colab for a max of 12hrs. 


In [ ]:
import jsonlines, json
import twarc
from twarc import Twarc

t = twarc.Twarc(consumer_key = "",
                consumer_secret = "",
                access_token = "",
                access_token_secret = "")
# Stores hydrated tweets here as jsonl objects
# Contains one json object per line
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
ids = []
with open(final_tweet_ids_filename, "r") as ids_file:
    ids = ids_file.read().split()
hydrated_tweets = []
ids_to_hydrate = set(ids)

# Looks at the output file for already hydrated tweets
if os.path.isfile(output_json_filename):
    with jsonlines.open(output_json_filename, "r") as reader:
        for i in reader.iter(type=dict, skip_invalid=True):
            # These tweets have already been hydrated. So remove them from ids_to_hydrate
            hydrated_tweets.append(i)
            ids_to_hydrate.remove(i["id_str"])
print("Total IDs: " + str(len(ids)) + ", IDs to hydrate: " + str(len(ids_to_hydrate)))
print("Hydrated: " + str(len(hydrated_tweets)))

count = len(hydrated_tweets)
start_index = count # The index from where tweets haven't been saved to the output_json_file
# Stores hydrated tweets to output_json_file every num_save iterations.
num_save  = 1000

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate):
    hydrated_tweets.append(tweet)
    count += 1
    # If num_save iterations have passed,
    if (count % num_save) == 0:
        # Open the output file
        # NOTE: Even if the code stops during IO, only tweets from the current iteration are lost.
        # Older tweets are preserved as the file is written in append mode.
        with jsonlines.open(output_json_filename, "a") as writer:
            print("Started IO")
            # Now write the tweets from start_index. The other tweets don't have to be written
            # as they were already written in a previous iteration or run.
            for hydrated_tweet in hydrated_tweets[start_index:]:
                writer.write(hydrated_tweet)
            print("Finished IO")
        print("Saved " + str(count) + " hydrated tweets.")
        # Now, since everything has been written. Reset start_index
        start_index = count
# There might be tweets unwritten in the last iteration if the count is not a multiple of num_tweets.
# In that case, just write out the remainder of tweets.
if count != start_index:
    print("Here with start_index", start_index)
    with jsonlines.open(output_json_filename, "a") as writer:
        for hydrated_tweet in hydrated_tweets[start_index:]:
           writer.write(hydrated_tweet)   

Total IDs: 52429, IDs to hydrate: 52429
Hydrated: 0
Started IO
Finished IO
Saved 1000 hydrated tweets.
Started IO
Finished IO
Saved 2000 hydrated tweets.
Started IO
Finished IO
Saved 3000 hydrated tweets.
Started IO
Finished IO
Saved 4000 hydrated tweets.
Started IO
Finished IO
Saved 5000 hydrated tweets.
Started IO
Finished IO
Saved 6000 hydrated tweets.
Started IO
Finished IO
Saved 7000 hydrated tweets.
Started IO
Finished IO
Saved 8000 hydrated tweets.
Started IO
Finished IO
Saved 9000 hydrated tweets.
Started IO
Finished IO
Saved 10000 hydrated tweets.
Started IO
Finished IO
Saved 11000 hydrated tweets.
Started IO
Finished IO
Saved 12000 hydrated tweets.
Started IO
Finished IO
Saved 13000 hydrated tweets.
Started IO
Finished IO
Saved 14000 hydrated tweets.
Started IO
Finished IO
Saved 15000 hydrated tweets.
Started IO
Finished IO
Saved 16000 hydrated tweets.
Started IO
Finished IO
Saved 17000 hydrated tweets.
Started IO
Finished IO
Saved 18000 hydrated tweets.
Started IO
Finished I

## Convert JSONL to CSV
Data is stored in  output_json_file from the previous code block. This now converts the jsonl .txt file into a csv file. Note that the column names required is stored as a list in the code.

Note that a few of the columns are actually json objects (for example, user or entities). You will have to clean these objects into 1D data.

In [ ]:
# Convert jsonl to csv
import csv, jsonlines
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
# These are the column name that are selected to be stored in the csv
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]
hydrated_tweets = []
# Reads the current tweets
with jsonlines.open(output_json_filename, "r") as reader:
    for i in reader.iter(type=dict, skip_invalid=True):
        hydrated_tweets.append(i)
# Writes them out
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset)
    d.writeheader()
    d.writerows(hydrated_tweets)